In [11]:
from hdf5libs import HDF5RawDataFile

import os
import daqdataformats
import detdataformats
from daqdataformats import FragmentType
from rawdatautils.unpack.daphne import *

import matplotlib.pyplot as plt
import os
import numpy as np

In [12]:
folder    = 'data0'
run       = 'run025156'
nskip     = 0
nrecords  = None #None if you want to run all the folder

geoids = [47244771330,51539738626,55834705922]
ep = 111
geoid_to_save = geoids[ep-111]
ch_to_save = range(48)
f_wf_ch = [open(f"pds_waveforms_{run}_wf_ep{ep}_ch{ch}.npy",'wb') for ch in ch_to_save]

det       = 'HD_PDS'

In [ ]:
files = os.listdir(f'/{folder}')

# Define a function to extract fragment information
def extract_fragment_info(frag):
    frag_id = str(frag).split(' ')[3][:-1]
    fragType = frag.get_header().fragment_type
    
    if fragType == FragmentType.kDAPHNE.value:  # For self trigger
        trigger = 'self_trigger'
        timestamps = np_array_timestamp(frag)
        adcs = np_array_adc(frag)
        channels = np_array_channels(frag)
    elif fragType == 13:  # For full_stream
        trigger = 'full_stream'
        timestamps = np_array_timestamp_stream(frag)
        adcs = np_array_adc_stream(frag)
        channels = np_array_channels_stream(frag)[0]
    
    return trigger, frag_id, channels, adcs, timestamps

# Iterate through files
for file in files:
    if file.endswith(('.json', '.json.copied')):
        continue  # Skip JSON files
        
    parts = file.split('_')
    if len(parts) <= 2 or parts[2] != run:
        continue  # Skip files not matching 'run'

    print(f'Reading {file}')
    filename  = f'/{folder}/{file}'
    h5_file = HDF5RawDataFile(filename)
    records = h5_file.get_all_record_ids()
    
    if nrecords is None: nrecord = len(records)
    # Iterate through records
    for r in records[nskip:nrecords]:
        
        frag = h5_file.get_frag(r, geoid_to_save)
        print(frag)
        trigger, frag_id, channels, adcs, timestamps = extract_fragment_info(frag)

        for ch_index, ch in enumerate(channels):
            if ch in ch_to_save:
                if ch == 24:
                    print(frag_id, adcs)
                doc_index = ch_to_save.index(ch)
                np.save(f_wf_ch[doc_index],np.array(adcs[ch_index]))
            

Reading np04hd_raw_run025156_0000_dataflow4_datawriter_0_20240426T093034.hdf5.copied
0x7f8a29a65530 [[8167 8167 8166 ... 8172 8172 8168]
 [8209 8210 8209 ... 8200 8200 8204]
 [8188 8192 8188 ... 8192 8189 8181]
 ...
 [8190 8188 8191 ... 8184 8186 8188]
 [8208 8206 8206 ... 8206 8206 8208]
 [8206 8207 8209 ... 8208 8210 8212]]
0x7f8a29a65530 [[8167 8167 8166 ... 8172 8172 8168]
 [8209 8210 8209 ... 8200 8200 8204]
 [8188 8192 8188 ... 8192 8189 8181]
 ...
 [8190 8188 8191 ... 8184 8186 8188]
 [8208 8206 8206 ... 8206 8206 8208]
 [8206 8207 8209 ... 8208 8210 8212]]
0x7f8a29024070 [[8199 8198 8197 ... 8194 8199 8200]
 [8201 8203 8201 ... 8204 8202 8200]
 [8172 8174 8169 ... 8172 8171 8170]
 ...
 [8185 8186 8184 ... 8183 8184 8187]
 [8184 8188 8191 ... 8183 8185 8183]
 [8207 8204 8199 ... 8202 8202 8203]]
0x7f8a29024070 [[8199 8198 8197 ... 8194 8199 8200]
 [8201 8203 8201 ... 8204 8202 8200]
 [8172 8174 8169 ... 8172 8171 8170]
 ...
 [8185 8186 8184 ... 8183 8184 8187]
 [8184 8188 8191 .